In [ ]:
import os
import pandas as pd
import matplotlib
import numpy as np
from typing import Tuple
from sklearn.metrics import roc_curve, precision_recall_curve, auc, roc_auc_score, average_precision_score
import matplotlib.pyplot as plt
from typing import Tuple
from plotnine import ggplot, aes, geom_point, geom_smooth, labs, theme_bw
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [ ]:
"""
CONTEXTO:
Enviar notificaciones push a clientes para animarlos a comprar un producto que ya han seleccionado previamente.

OBJETIVO:
Desarrollar un modelo de aprendizaje automático que dado un usuarioo y un producto prediga si el usuario
compraría el produco si estuviera comprando con nosotros en ese momento.

DATOS:
Usar la base de datos feature_frame.csv filtrando sólo los pedidos de al menos 5 productos. En este caso,
uso directamente feature_frame_filtered.csv que filtra dichos pedidos.
"""

In [ ]:
# CONSTRUCCIÓN DEL MODELO LINEAL PREDICTIVO

In [ ]:
#1. Se cargan los datos
BASE_DIR = os.getcwd()  # obtiene el directorio actual
DATA_PATH = os.path.join(BASE_DIR, "data", "feature_frame_filtered.csv")

print("Cargando datos desde:", DATA_PATH)
df = pd.read_csv(DATA_PATH)
print("Datos cargados correctamente:", df.shape)

In [ ]:
df.head # para ver las primeras filas

In [ ]:
df.info() # para ver tipo de datos, si hay nulos

In [ ]:
# Clasificación de columnas
info_col = ["variant_id", "order_id", "user_id", "created_at",
            "order_date"]  # id y fechas (son distintos de cada pedido)
label_col = "outcome"  # variable objetivo (y)
features_col = [col for col in df.columns if col not in info_col + [label_col]]  # resto de columnas

categorical_col = ["predict_type", "vendor"]  # columnas categóricas (de las features)
binary_col = ["ordered_before", "abandoned_before", "active_snoozed",
              "set_as_regular"]  # columnas binarias (de las features)
numerical_col = [col for col in features_col if col not in categorical_col + binary_col]  # resto (de las features)

In [ ]:
# 2. Proceso de entrenamiento, validación y test
# NOTA: Se tiene que hacer un SPLIT TEMPORAL, para evitar INFORMATION LEAKAGE. Es decir, respeto el orden temportal de los datos. Quiero que el proceso de entretanmiento, validación sean lo más cercano posible al proceso de producción.

In [ ]:
# En la gráfica de los perdidos por días, los dividimos en 3 columnas y cada parte se encarga de hacer train, val y test.
daily_orders = df.groupby("order_date").order_id.nunique()  # pedidos diarios
daily_orders.head()  # primeras filas

df["order_date"] = pd.to_datetime(df["order_date"])  # eje x, convertir a datetime si no lo es
daily_orders = df.groupby("order_date").order_id.nunique()  # agrupo por fecha

plt.plot(daily_orders, label="daily_orders")  # presentación
plt.title("Pedidos Diarios")
# De este modo, una orden (pedido) o está en train, o en validación o en test.
# Entreno con el 70% de los datos y valido con el 90%.

In [ ]:
# Para ello, creo una función que me defina el nº de pedidos acumulativos.
orders_acum = daily_orders.cumsum() / daily_orders.sum()

plt.plot(orders_acum, label="orders_acum")  # represento (tiene que dar 1 la suma obv)
plt.title("Suma acumulativa de pedidos")
# Realizo la división (cut off).
train_val_cutoff = orders_acum[orders_acum <= 0.7].idxmax()
val_test_cutoff = orders_acum[orders_acum <= 0.9].idxmax()

print("Entrenamiendo desde:", orders_acum.index.min())
print("Entranmiento hasta:", train_val_cutoff)
print("Validación hasta:", val_test_cutoff)
print("Test hasta:", orders_acum.index.max())

In [ ]:
# Comprobaciones
train_df = df[df.order_date <= train_val_cutoff]
val_df = df[(df.order_date > train_val_cutoff) & (df.order_date <= val_test_cutoff)]
test_df = df[df.order_date > val_test_cutoff]

In [ ]:
# BASELINE: Siempre hay que hacer una baseline, es decir, un modelo sencillo que nos de métricas que queremos mejorar para producción.
# En este caso, como baseline uso la variable product_popularity, ¿cómo de popular es un producto?. La probabilidad de vender un producto depende de su popularidad.
# El modelo (baseline) predice la prob. de que un usuario compre un producto si recibe una notificación push. Sin embargo:
# Si envío una notificación y el usuario no compra, le molesta → coste negativo (falso positivo).
# Si no envío y el usuario habría comprado, pierdo una venta → coste de oportunidad (falso negativo).
# Por tanto, no debo enviar notificaciones a todos los usuarios con prob > 0.5, sino solo cuando la ganancia esperada sea positiva.

In [ ]:
def plot_metrics(model_name, y_pred, y_test):
    """
    Dibuja curvas Precision–Recall y ROC para un modelo binario.

    Parámetros
    ----------
    model_name : str
        Nombre del modelo para la leyenda.
    y_pred : array-like
        Probabilidades predichas (no etiquetas).
    y_test : array-like
        Etiquetas reales (0/1).

    Retorna
    -------
    fig, ax : matplotlib.figure.Figure, matplotlib.axes._subplots.AxesSubplot
        Figura y ejes de los subplots.
    """

    # ==========================
    # Curva Precision–Recall
    # ==========================
    recall_, precision_, _ = precision_recall_curve(y_test, y_pred)
    avg_precision = average_precision_score(y_test, y_pred)  # PR-AUC estable

    # ==========================
    # Curva ROC
    # ==========================
    fpr, tpr, _ = roc_curve(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)

    # ==========================
    # Crear figura con dos subplots
    # ==========================
    fig, ax = plt.subplots(1, 2, figsize=(14, 6))

    # --- Precision–Recall ---
    ax[0].plot(recall_, precision_, color='royalblue',
               label=f"{model_name}\nAP={avg_precision:.3f}")
    ax[0].set_xlabel("Recall")
    ax[0].set_ylabel("Precision")
    ax[0].set_title("Curva Precision–Recall")
    ax[0].set_xlim(0, max(recall_) * 1.05)  # zoom dinámico
    ax[0].set_ylim(0, max(precision_) * 1.05)
    ax[0].legend(loc="upper right", fontsize=9)
    ax[0].grid(True, linestyle='--', alpha=0.5)

    # --- ROC ---
    ax[1].plot(fpr, tpr, color='darkorange',
               label=f"{model_name}\nAUC={roc_auc:.3f}")
    ax[1].plot([0, 1], [0, 1], '--', color='gray', alpha=0.6)
    ax[1].set_xlabel("False Positive Rate")
    ax[1].set_ylabel("True Positive Rate")
    ax[1].set_title("Curva ROC")
    ax[1].legend(loc="lower right", fontsize=9)
    ax[1].grid(True, linestyle='--', alpha=0.5)

    plt.tight_layout()
    return fig, ax


fig, ax = plot_metrics(
    "Popularity baseline",
    y_pred=val_df["global_popularity"],
    y_test=val_df[label_col]
)

plt.show()  # ✅ Obligatorio en PyCharm

In [ ]:
# Ahora realizamos un entrenamiento con 2 modelos lineales (L1, L2) y diferentes hiperparámetros.
# Preprocesamiento + pipeline + entrenamiento + evaluación.

In [ ]:
# Preprocesamiento
categorical_col = ["product_type", "vendor"]
binary_col = ["ordered_before", "abandoned_before", "active_snoozed", "set_as_regular"]
numerical_col = [col for col in features_col if col not in categorical_col + binary_col]

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_col),  # standardscaler -> media 0 sd 1
        ("cat", OneHotEncoder(handle_unknown='ignore'), categorical_col)
    ],
    remainder='passthrough'  # columnas binarias se mantienen tal cual
)

# Conjuntos
X_train = train_df[features_col]
y_train = train_df[label_col]
X_val = val_df[features_col]
y_val = val_df[label_col]

In [ ]:
# Entrenamiento y papeline
predictions = {}
estimators = {}
scores = {}

# Valores de C a explorar
C_values = [0.01, 0.1, 1]

for penalty in ["l1", "l2"]:
    print(f"\n=== Entrenando modelo con penalización {penalty} ===")

    # Definir el modelo base según la penalización
    solver = "liblinear" if penalty == "l1" else "lbfgs"
    log_reg = LogisticRegression(penalty=penalty, solver=solver, max_iter=1000)

    # GridSearchCV para encontrar el mejor C
    grid = GridSearchCV(log_reg, param_grid={"C": C_values}, scoring="average_precision", cv=5)
    grid.fit(X_train, y_train)

    # Mejor modelo y C encontrado
    best_C = grid.best_params_["C"]
    best_score = grid.best_score_
    print(f"Mejor C para {penalty}: {best_C}, score medio CV={best_score:.4f}")

    # Predicciones sobre el conjunto de validación
    y_pred_val = grid.predict_proba(X_val)[:, 1]
    ap_val = average_precision_score(y_val, y_pred_val)
    print(f"Average Precision en validación: {ap_val:.4f}")

    # Guardar resultados
    predictions[penalty] = y_pred_val
    estimators[penalty] = grid.best_estimator_
    scores[penalty] = ap_val


In [ ]:
# Gráficas
penalties = list(predictions.keys())

# ---------------------
# CURVA ROC
# ---------------------
plt.figure(figsize=(8, 6))

for penalty in penalties:
    y_pred = predictions[penalty]
    fpr, tpr, _ = roc_curve(y_val, y_pred)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2, label=f"{penalty.upper()} (AUC = {roc_auc:.3f})")

plt.plot([0, 1], [0, 1], 'k--', lw=1)
plt.xlabel("Tasa de Falsos Positivos (1 - Especificidad)")
plt.ylabel("Tasa de Verdaderos Positivos (Sensibilidad)")
plt.title("Curvas ROC para diferentes penalizaciones")
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# ---------------------
# CURVA PRECISIÓN–RECALL
# ---------------------
plt.figure(figsize=(8, 6))

for penalty in penalties:
    y_pred = predictions[penalty]
    precision, recall, _ = precision_recall_curve(y_val, y_pred)
    ap = average_precision_score(y_val, y_pred)
    plt.plot(recall, precision, lw=2, label=f"{penalty.upper()} (AP = {ap:.3f})")

plt.xlabel("Recall (Sensibilidad)")
plt.ylabel("Precisión")
plt.title("Curvas Precisión–Recall para diferentes penalizaciones")
plt.legend(loc="lower left")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# No entiendo por qué no las dibujas. Me falta decir con qué modelo me quedo con cual no y por qué.